In [116]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torchsummary import summary
from torcheval.metrics import BinaryPrecisionRecallCurve
import torchmetrics
from tqdm.notebook import tqdm
from torchmetrics import R2Score

%matplotlib inline

In [50]:
df=pd.read_csv('datasets/data.csv')
df[['q_mu', 'q_nu1', 'q_nu2', 'e_nu_1', 'e_nu_2']]

,q_mu,q_nu1,q_nu2,e_nu_1,e_nu_2
0,0.550377,0.164911,0.008094,0.01904,0.17794
1,0.564295,0.167629,0.006639,0.01672,0.20868
2,0.564179,0.164110,0.007052,0.01337,0.20442
3,0.573555,0.167174,0.006663,0.01637,0.18453
4,0.569296,0.169658,0.006823,0.01783,0.11478
...,...,...,...,...,...
184845,0.566484,0.172819,0.008164,0.02085,0.25589
184846,0.572031,0.170474,0.007907,0.01592,0.23668
184847,0.569204,0.173147,0.007366,0.01590,0.24216
184848,0.570114,0.168854,0.007378,0.01384,0.24447


In [3]:
data=np.log(df['e_mu_current'])
data_std=np.std(data)
data_mean=np.mean(data)

labels=[]

for point in data:
    if point >=2*data_std+data_mean:
        labels.append(1)
    else:
        labels.append(0)

labels=np.array(labels)

In [77]:
class BoxDataset(Dataset):
    
    @classmethod
    def __init__(self, df_path):
        self.data=pd.read_csv(df_path)
        self.x = self.data[['q_mu', 'q_nu1', 'q_nu2', 'e_nu_1', 'e_nu_2']].values
        labels=[]
        self.matrices=[]
        
        log_data=np.log(self.data['e_mu_current'])
        data_std=np.std(log_data)
        data_mean=np.mean(log_data)

        for i, point in enumerate(log_data[5:-1]):
            if point >=2*data_std+data_mean:
                labels.append(1)
            else:
                labels.append(0)
            self.matrices.append(self.x[i:5+i,:])
            
        self.labels=np.array(labels)
        
        self.labels = torch.FloatTensor(self.labels)
        self.matrices = torch.FloatTensor(self.matrices)
    def __len__(self):
        return len(self.matrices)

    def __getitem__(self, idx):
        return {'matrices':self.matrices[idx], 'labels':self.labels[idx]}

In [121]:
dataset=BoxDataset(df_path='datasets/data.csv')

val_sptit = 0.2
batch_size = 512

train_len = len(dataset) - int(len(dataset) * val_sptit)
test_len = int(len(dataset) * val_sptit)

train_set, val_set = torch.utils.data.random_split(dataset, [train_len, test_len])

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True,
                          drop_last=True)
test_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False,
                         drop_last=True)

In [127]:
class Net(nn.Module):
    def __init__(self,in_channels=1):
        super(Net, self).__init__()
        
        self.relu = nn.LeakyReLU()

        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=16,kernel_size=(3, 3))
        self.batch_norm1 = nn.BatchNorm2d(16)
        self.maxpool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32,kernel_size=(3, 3))
        self.batch_norm2 = nn.BatchNorm2d(32)
        self.maxpool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64,kernel_size=(3, 3))
        self.batch_norm3 = nn.BatchNorm2d(64)
        self.maxpool3 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128,kernel_size=(2, 2))
        self.batch_norm4 = nn.BatchNorm2d(128)
        self.maxpool4 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

        self.fc1_1 = nn.Linear(in_features=32, out_features=256)
        self.anomaly = nn.Linear(in_features=256, out_features=1)


        for tensor in [self.conv1, self.conv2, self.conv3, self.conv4, self.fc1_1, self.anomaly]:
            torch.nn.init.kaiming_uniform_(tensor.weight)

    def forward(self, x):
        # x = torch.fft.fft2(x).imag
        # print(x.shape)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.batch_norm1(x)
        # x = self.maxpool1(x)

        # x = torch.fft.fft2(x).real

        x = self.conv2(x)
        x = self.relu(x)
        x = self.batch_norm2(x)
        # x = self.maxpool2(x)

        # x = torch.fft.fft2(x).real
#         x = self.conv3(x)
#         x = self.relu(x)
#         x = self.batch_norm3(x)
#         # x = self.maxpool3(x)

#         x = self.conv4(x)
#         x = self.relu(x)
#         x = self.batch_norm4(x)
        # x = self.maxpool3(x)

        x = torch.flatten(x, 1)

        output1_1 = self.fc1_1(x)
        anomaly = self.anomaly(output1_1)
        anomaly=nn.Sigmoid()(anomaly)

        return anomaly

In [128]:
model = Net()

res = model.cuda()
summary(model, (1, 5, 5))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [-1, 16, 3, 3]             160
         LeakyReLU-2             [-1, 16, 3, 3]               0
       BatchNorm2d-3             [-1, 16, 3, 3]              32
            Conv2d-4             [-1, 32, 1, 1]           4,640
         LeakyReLU-5             [-1, 32, 1, 1]               0
       BatchNorm2d-6             [-1, 32, 1, 1]              64
            Linear-7                  [-1, 256]           8,448
            Linear-8                    [-1, 1]             257
Total params: 13,601
Trainable params: 13,601
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.05
Estimated Total Size (MB): 0.06
----------------------------------------------------------------


In [129]:
log=False
# log=True
if log==True:
    experiment = Experiment(
        api_key="5skWtD5KzMcU6rVhVNOLfQZ6E",
        project_name="tracks-nn",
        workspace="dkagramanyan",
    )

device = 'cuda'
# device = 'cpu'

model.train()
model.to(device)

learning_rate = 0.001
# learning_rate = 0.1 * (batch_size / 256)
print('lr=', learning_rate)

bce_loss = torch.nn.BCEWithLogitsLoss()
precision_recall_1 = BinaryPrecisionRecallCurve()

acc_metric=torchmetrics.classification.MulticlassAccuracy(num_classes=5)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)

epochs_cnt = 10000
tbar = tqdm(range(epochs_cnt), leave=True, desc='?')


for epoch in tbar:

    for i, frames_series in enumerate(train_loader):

        matrices = frames_series['matrices'].to(device)
        anomaly_true = frames_series['labels'].to(device)
        m=torch.reshape(matrices, (batch_size, 1, 5, 5))

        anomaly_output = model(m)
        
        # BCE change point classification
        loss = bce_loss(torch.squeeze(anomaly_output), anomaly_true).to(device)

        # Precision@recall metrics
        precision, recall, threshold=precision_recall_1.update(anomaly_output.squeeze(), anomaly_true).compute()
        precision_recall_1_metrics=precision[torch.where(recall==1)[0][-1]]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if log==True:
            experiment.log_metric('Train loss', loss.item())
            experiment.log_metric('Train Precision@Recall=1', precision_recall_1_metrics)
            
    if epoch % 5 == 0:
        msg = f'Train Epoch: {epoch + 1}/{epochs_cnt}\tStep: {i + 1}/{n_total_steps}\tLoss: {loss.item():.5f}\t\tPrecision@Recall=1: {precision_recall_1_metrics:.4f}'

        # print(msg)



    if epoch % 5 == 0:
        with torch.no_grad():
            r2score = R2Score().to(device)

            loss_list=[]
            rotate_loss_list=[]
            change_point_type_loss_list=[]
            change_point_loss_list=[]

            precision_recall_1_metrics_list=[]
            f_1_metrics_list=[]



            for j, test_frames_series in enumerate(test_loader):
                matrices = test_frames_series['matrices'].to(device)
                anomaly_true = test_frames_series['labels'].to(device)
                m=torch.reshape(matrices, (batch_size, 1, 5, 5))

                anomaly_output = model(m)

                # BCE change point classification
                loss = bce_loss(torch.squeeze(anomaly_output), anomaly_true).to(device)

                # Precision@recall metrics
                precision, recall, threshold=precision_recall_1.update(anomaly_output.squeeze(), anomaly_true).compute()
                precision_recall_1_metrics=precision[torch.where(recall==1)[0][-1]]
                precision_recall_1_metrics_list.append(precision_recall_1_metrics)
                
                f_1_metrics=f1_score(torch.argmax(nn.Softmax(dim=1)(misalignment_type_output),dim=1).detach().cpu().numpy().flatten(),
                                     torch.argmax(nn.Softmax(dim=1)(misalignment_type_output),dim=1).detach().cpu().numpy().flatten(), average='macro')
                f_1_metrics_list.append(f_1_metrics)


                loss_list.append(loss)

            loss_v=torch.mean(torch.FloatTensor(loss_list))

            precision_recall_1_metrics_v=torch.mean(torch.FloatTensor(precision_recall_1_metrics_list)).to(device)
            f_1_metrics_v=torch.mean(torch.FloatTensor(f_1_metrics_list)).to(device)



            msg = f'\n Test Loss: {loss_v:.4f}\tPrecision@Recall=1: {precision_recall_1_metrics_v:.4f}t\Anomaly type F_1: {f_1_metrics_v:.6f}'

            print(msg)
            if log==True:
                experiment.log_metric('Test loss', loss_v)
                experiment.log_metric('Test Precision@Recall=1', precision_recall_1_metrics_v)
                experiment.log_metric('Test Misalignment point BCE', change_point_loss)
                experiment.log_metric('Test Misalignment type F_1', f_1_metrics_v)



lr= 0.001


?:   0%|          | 0/10000 [00:00<?, ?it/s]

NameError: name 'f1_score' is not defined

In [ ]:
f1_score(torch.argmax(nn.Softmax(dim=1)(misalignment_type_output),dim=1).detach().cpu().numpy().flatten(),
         torch.argmax(nn.Softmax(dim=1)(misalignment_type_output),dim=1).detach().cpu().numpy().flatten(),
         average='macro')
                

In [126]:
anomaly_true

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [125]:
anomaly_output

tensor([[-6.7303],
        [-4.6913],
        [-5.3316],
        [-3.7295],
        [-5.7629],
        [-5.4130],
        [-6.5191],
        [-3.8755],
        [-4.4416],
        [-5.5048],
        [-5.8610],
        [-5.1116],
        [-4.6043],
        [-5.5485],
        [-4.7396],
        [-4.4899],
        [-4.3086],
        [-5.2123],
        [-4.2094],
        [-4.5325],
        [-5.3925],
        [-5.4343],
        [-5.9019],
        [-5.0039],
        [-5.6280],
        [-5.3549],
        [-4.6506],
        [-5.4723],
        [-3.8709],
        [-4.4635],
        [-4.3633],
        [-4.0836],
        [-4.7503],
        [-6.4468],
        [-4.8545],
        [-4.2444],
        [-3.8263],
        [-5.6282],
        [-5.1895],
        [-5.7484],
        [-3.2937],
        [-4.4635],
        [-5.7864],
        [-4.2709],
        [-3.7824],
        [-4.5130],
        [-4.7942],
        [-5.7483],
        [-4.7901],
        [-4.2366],
        [-3.2711],
        [-5.3370],
        [-5.

In [101]:
m.shape

torch.Size([128, 1, 5, 5])